In [ ]:
import warnings
warnings.filterwarnings('ignore')

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
device = "mps"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map=device,
    torch_dtype="auto",
    trust_remote_code=False,
)

In [ ]:
prompt = "The capital of France is"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Get the output of the model before the lm_head and calculate the output of the lm_head ourselves
model_output = model.model(input_ids)
lm_head_output = model.lm_head(model_output[0])
print("Output size:", lm_head_output.shape) # gives a vector of probabilities for each input token

# Find and print top-10
values, indices = torch.topk(lm_head_output[0, -1], k=10) # -1 because we're only interested in the last token's output
print("Values (probabilities):", values)
print("Corresponding indices: ", indices)
for index in indices:
    print(tokenizer.decode(index), end=" ")